<a href="https://colab.research.google.com/github/Cru1zzz3/python-parallel-programming-cookbook/blob/main/Python_Parallel_Programming_(Lab_5)_Udartsev_Stanislav.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Using Celery to distribute tasks**

Celery- это фреймворк Python, используемый для управления распределенными задачами, следуя объектно-ориентированному
подходу для  промежуточного ПО. Его основная особенность заключается в обработке множества небольших задач и распределении
их по большому количеству вычислительных узлов. Результат каждой задачи перерабатывается для составления общего решения.

In [2]:
!pip install  celery

     |████████████████████████████████| 404 kB 25.7 MB/s 
     |████████████████████████████████| 97 kB 6.7 MB/s 
     |████████████████████████████████| 188 kB 71.5 MB/s 
     |████████████████████████████████| 89 kB 7.3 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
flask 1.1.4 requires click<8.0,>=5.1, but you have click 8.0.3 which is incompatible.


In [3]:
# !/usr/bin/sh
!sudo apt-get install curl gnupg apt-transport-https -y

!wget https://packages.erlang-solutions.com/ubuntu/erlang_solutions.asc

!sudo apt-key add erlang_solutions.asc

!apt update && apt install esl-erlang


## Install rabbitmq-server and its dependencies
!sudo apt-get install rabbitmq-server -y 

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.16).
gnupg is already the newest version (2.2.4-1ubuntu1.4).
gnupg set to manually installed.
The following NEW packages will be installed:
  apt-transport-https
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 4,348 B of archives.
After this operation, 154 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 apt-transport-https all 1.6.14 [4,348 B]
Fetched 4,348 B in 0s (27.2 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back

In [4]:
!pip install -U flower

     |████████████████████████████████| 457 kB 29.0 MB/s 


In [5]:
!celery --version

5.2.1 (dawn-chorus)


In [6]:
!service rabbitmq-server start
!rabbitmqctl status

 * Starting RabbitMQ Messaging Server rabbitmq-server
   ...done.
Status of node rabbit@021733a5f0fa
[{pid,4514},
 {running_applications,
     [{rabbit,"RabbitMQ","3.6.10"},
      {mnesia,"MNESIA  CXC 138 12","4.15.3"},
      {ranch,"Socket acceptor pool for TCP protocols.","1.3.0"},
      {ssl,"Erlang/OTP SSL application","8.2.3"},
      {public_key,"Public key infrastructure","1.5.2"},
      {asn1,"The Erlang ASN1 compiler version 5.0.4","5.0.4"},
      {rabbit_common,
          "Modules shared by rabbitmq-server and rabbitmq-erlang-client",
          "3.6.10"},
      {xmerl,"XML parser","1.3.16"},
      {crypto,"CRYPTO","4.2"},
      {os_mon,"CPO  CXC 138 46","2.4.4"},
      {compiler,"ERTS  CXC 138 10","7.1.4"},
      {syntax_tools,"Syntax tools","2.1.4"},
      {sasl,"SASL  CXC 138 11","3.1.1"},
      {stdlib,"ERTS  CXC 138 10","3.4.3"},
      {kernel,"ERTS  CXC 138 10","5.4.1"}]},
 {os,{unix,linux}},
 {erlang_version,
     "Erlang/OTP 20 [erts-9.2] [source] [64-bit] [smp:2:2] [ds

**How to create a task with Celery**

Задачи(Tasks) являются строительными блоками приложений для сельдерея.

Задача - это класс, который может быть создан из любого вызываемого объекта. Он выполняет двойную роль, поскольку определяет, что происходит при вызове задачи (отправляет сообщение), так и то, что происходит, когда обрабатывающий узел получает это сообщение.

In [7]:
def addTask():
  """
from celery import Celery

app = Celery('addTask',broker='amqp://guest@localhost//')

@app.task
def add(x, y):
  return x + y
  """

# addTask.py
with open('/content/addTask.py', 'w') as f:
  f.write(addTask.__doc__)


In [8]:
import addTask
if __name__ == '__main__':
  result = addTask.add.delay(5,5)

In [9]:
!celery -A addTask worker --loglevel=info

/usr/local/lib/python3.7/dist-packages/celery/platforms.py:841: SecurityWarning: You're running the worker with superuser privileges: this is
absolutely not recommended!

Please specify a different user using the --uid option.

User information: uid=0 euid=0 gid=0 egid=0

  uid=uid, euid=euid, gid=gid, egid=egid,
 
 -------------- celery@021733a5f0fa v5.2.1 (dawn-chorus)
--- ***** ----- 
-- ******* ---- Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic 2021-11-25 18:55:33
- *** --- * --- 
- ** ---------- [config]
- ** ---------- .> app:         addTask:0x7f0c06e47b10
- ** ---------- .> transport:   amqp://guest:**@localhost:5672//
- ** ---------- .> results:     disabled://
- *** --- * --- .> concurrency: 2 (prefork)
-- ******* ---- .> task events: OFF (enable -E to monitor tasks in this worker)
--- ***** ----- 
 -------------- [queues]
                .> celery           exchange=celery(direct) key=celery
                

[tasks]
  . addTask.add

[2021-11-25 18:55:33,690: INFO/MainProce

**Scientific computing with SCOOP**

SCOOP - это библиотека, которая в основном используется для научных вычислительных задач. В качестве примера в учебнике рассмотрено параллальное выполнение алгоритма Монте-Карло для вычисления числа Пи, требующий больших вычислительных затрат.

In [10]:
!pip install SCOOP

     |████████████████████████████████| 603 kB 28.7 MB/s 
  Created wheel for SCOOP: filename=scoop-0.7.1.1-py3-none-any.whl size=72145 sha256=297c9f79feff7d5714bc8f9fccdc2bf3267eaffea8c84b89262d306e26965b6f
  Stored in directory: /root/.cache/pip/wheels/24/19/e9/6e3e7c0323cc36bf1e4993d69b2db27d6b4e806ed57d411f44
Successfully built SCOOP


В примере из учебника обнаружены следуюшие опечатки: 

*   NameError: name 'evaluate_points_in_circle' is not defined. Должна быть функция 'evaluate_number_of_points_in_unit_circle'
*   NameError: name 'Taskresult' is not defined. Должна быть переменная 'Taskresult' вместо 'taskresult'




In [11]:
def pi_calculus_with_montecarlo_method():
  """
import math
from random import random
from scoop import futures
from time import time

def evaluate_number_of_points_in_unit_circle(attempts):
  points_fallen_in_unit_disk = 0
  for i in range (0,attempts):
    x = random()
    y = random()
    radius = math.sqrt(x*x + y*y)
    #the test is ok if the point fall in the unit circle
    if radius < 1 :
      #if ok the number of points in a disk is increased
      points_fallen_in_unit_disk = points_fallen_in_unit_disk + 1
  return points_fallen_in_unit_disk

def pi_calculus_with_Montecarlo_Method(workers, attempts):
 print("number of workers %i - number of attempts %i" %(workers,attempts))
 bt = time()
 #in this point we call scoop.futures.map function
 #the evaluate_number_of_points_in_unit_circle function is executed in an asynchronously way
 #and several call this function can be made concurrently
 evaluate_task =  futures.map(evaluate_number_of_points_in_unit_circle, [attempts] * workers)
 Taskresult= sum(evaluate_task)
 print ("%i points fallen in a unit disk after " %(Taskresult/attempts))
 piValue = (4. * Taskresult/ float(workers * attempts))

 computationalTime = time() - bt
 print("value of pi = " + str(piValue))
 print ("error percentage = " +  str((((abs(piValue - math.pi)) * 100) / math.pi)))
 print("total time: " + str(computationalTime))

if __name__ == "__main__":
  for i in range (1,4):
    #let's fix the numbers of workers...only two,
    #but it could be much greater
    pi_calculus_with_Montecarlo_Method(i*1000, i*1000)
    print(" ")
  """

# pi_calculus_with_montecarlo_method.py
with open('/content/pi_calculus_with_montecarlo_method.py', 'w') as f:
  f.write(pi_calculus_with_montecarlo_method.__doc__)
  
!python -m scoop pi_calculus_with_montecarlo_method.py


[2021-11-25 18:56:31,953] launcher  INFO    SCOOP 0.7 1.1 on linux using Python 3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0], API: 1013
[2021-11-25 18:56:31,953] launcher  INFO    Deploying 2 worker(s) over 1 host(s).
[2021-11-25 18:56:31,953] launcher  INFO    Worker distribution: 
[2021-11-25 18:56:31,953] launcher  INFO       127.0.0.1:	1 + origin
number of workers 1000 - number of attempts 1000
784 points fallen in a unit disk after 
value of pi = 3.139792
error percentage = 0.05731658392235075
total time: 0.8658866882324219
 
number of workers 2000 - number of attempts 2000
1570 points fallen in a unit disk after 
value of pi = 3.141516
error percentage = 0.0024399595442562965
total time: 2.356513500213623
 
number of workers 3000 - number of attempts 3000
2355 points fallen in a unit disk after 
value of pi = 3.141228888888889
error percentage = 0.011578990054244743
total time: 4.511167049407959
 
[2021-11-25 18:56:40,227] launcher  (127.0.0.1:42021) INFO    Root process i

**Handling map functions with SCOOP**

С помощью библиотеки SCOOP можно определить более одной функции map, они позволяют выполнять асинхронные
вычисления, которые могут быть переданы на обрабатыющие узлы.
В данном примере сравниваются функции futures.mapReduce из библиотеки SCOOP и встроеная функция map стандарнтной библиотеки. Из результатов работы можем видеть, что использования механизма mapReduce позволяет уменьшить время выполнения почти в 2 раза.

In [12]:
def map_reduce():
  """
import operator
import time
from scoop import futures

def simulateWorkload(inputData):
  time.sleep(0.01)
  return sum(inputData)

def CompareMapReduce():
  mapScoopTime = time.time()
  res = futures.mapReduce(simulateWorkload, operator.add, list([a] * a for a in range(1000)),)
  mapScoopTime = time.time() - mapScoopTime

  print("futures.map in SCOOP executed in {0:.3f}s with result:{1}".format(mapScoopTime, res))

  mapPythonTime = time.time()
  res = sum(map(simulateWorkload,list([a] * a for a in range(1000))))
  mapPythonTime = time.time() - mapPythonTime
  print("map Python executed in: {0:.3f}s with result: {1}".format(mapPythonTime,res))

if __name__ == '__main__':
  CompareMapReduce()
  """

# pi_calculus_with_montecarlo_method.py
with open('/content/map_reduce.py', 'w') as f:
  f.write(map_reduce.__doc__)
  
!python -m scoop map_reduce.py

[2021-11-25 18:56:40,432] launcher  INFO    SCOOP 0.7 1.1 on linux using Python 3.7.12 (default, Sep 10 2021, 00:21:48) [GCC 7.5.0], API: 1013
[2021-11-25 18:56:40,433] launcher  INFO    Deploying 2 worker(s) over 1 host(s).
[2021-11-25 18:56:40,433] launcher  INFO    Worker distribution: 
[2021-11-25 18:56:40,433] launcher  INFO       127.0.0.1:	1 + origin
futures.map in SCOOP executed in 5.569s with result:332833500
map Python executed in: 10.183s with result: 332833500
[2021-11-25 18:56:56,744] launcher  (127.0.0.1:34509) INFO    Root process is done.
[2021-11-25 18:56:56,745] launcher  (127.0.0.1:34509) INFO    Finished cleaning spawned subprocesses.


**Remote Method Invocation with Pyro4**

In [27]:
!pip install Pyro4

Для корректной работы программы необходимо использовать декоратор @Pyro4.expose, иначе возникает следующая ошибка:

```
doesn't expose any methods or attributes. Did you forget setting @expose on them?
```

In [2]:
def server():
  """
import Pyro4

@Pyro4.expose
class Server(object):
  def welcomeMessage(self, name):
    return ("Hi welcome " + str (name))

  def shutdown(self):
    print('shutting down...')
    self.running = False

def startServer():
  server = Server()
  daemon = Pyro4.Daemon()
  ns = Pyro4.locateNS()
  uri = daemon.register(server)
  ns.register("server", uri)
  print("Ready. Object uri =", uri)
  daemon.requestLoop()
  daemon.close()

if __name__ == "__main__":
  startServer()
  """

# server.py
with open('/content/server.py', 'w') as f:
  f.write(server.__doc__)

def client():
  """
import Pyro4

uri = input("What is the Pyro uri of the greeting object? ").strip()
name = input("What is your name? ").strip()
server = Pyro4.Proxy("PYRONAME:server")
print(server.welcomeMessage(name))
server.shutdown()
server._pyroRelease()
print('client exiting')
  """

# client.py
with open('/content/client.py', 'w') as f:
  f.write(client.__doc__)


Немного схитрим и поставим таймаут в 30 секунд на выполнение процессов для корректного отображения лабораторной, т.к. один из процессов выполняется бесконечно долго.

In [4]:
!timeout -s KILL 30 python -m Pyro4.naming & timeout -s KILL 30 python server.py & timeout -s KILL 30 python client.py

What is the Pyro uri of the greeting object? Not starting broadcast server for localhost.
NS running on localhost:9090 (127.0.0.1)
URI = PYRO:Pyro.NameServer@localhost:9090
Ready. Object uri = PYRO:obj_09235af1e69e40ae919531a94ade2eec@localhost:41363
PYRO:obj_09235af1e69e40ae919531a94ade2eec@localhost:41363
What is your name? Stanislav
Hi welcome Stanislav
shutting down...
client exiting


**Chaining objects with Pyro4**

Возникает ошибка при выполнении кода из учебника:
```
 NameError: name 'chain' is not defined
```
Опечатка в функции server_2 и server_3

```
obj = chain.chainTopology(this_server, next)
```
Должно быть:
```
obj = chainTopology.Chain(this_server, next)
```

Pyro4.errors.NamingError: unknown name: example.chain.A

Возникает ошибка при выполнении функции кода из учебника:
```
 obj = Pyro4.core.Proxy("PYRONAME:example.chain.A")
```
Опечатка в функции client

```
obj = Pyro4.core.Proxy("PYRONAME:example.chain.A")
```
Должно быть:
```
obj = Pyro4.core.Proxy("PYRONAME:example.chainTopology.1")
```



Возникает ошибка при выполнении функции кода из учебника:
```
Pyro4.errors.NamingError: unknown name: example.chainTopology.1
```
Опечатка в функции client

```
Pyro4.errors.NamingError: unknown name: example.chainTopology.1
```
Должно быть:
```
self.next = Pyro4.core.Proxy("PYRONAME:example.chainTopology." + self.nextName)

```



In [27]:
def client():
  """
from __future__ import print_function
import Pyro4

obj = Pyro4.core.Proxy("PYRONAME:example.chainTopology.1")
print("Result=%s" % obj.process(["hello"]))
  """
# client.py
with open('/content/client.py', 'w') as f:
  f.write(client.__doc__)

def server_1():
  """
from __future__ import print_function
import Pyro4
import chainTopology

this = "1"
next = "2"
servername = "example.chainTopology." + this
daemon = Pyro4.core.Daemon()
obj = chainTopology.Chain(this, next)
uri = daemon.register(obj)
ns = Pyro4.locateNS()
ns.register(servername, uri)

# enter the service loop.
print("server_%s started " % this)
daemon.requestLoop()
  """

# server_1.py
with open('/content/server_1.py', 'w') as f:
  f.write(server_1.__doc__)



def server_2():
  """
from __future__ import print_function
import Pyro4
import chainTopology

this = "2"
next = "3"
servername = "example.chainTopology." + this
daemon = Pyro4.core.Daemon()
obj = chainTopology.Chain(this, next)
uri = daemon.register(obj)
ns = Pyro4.locateNS()
ns.register(servername, uri)
# enter the service loop.
print("server_%s started " % this)
daemon.requestLoop()
  """

# server_2.py
with open('/content/server_2.py', 'w') as f:
  f.write(server_2.__doc__)


def server_3():
  """
from __future__ import print_function
import Pyro4
import chainTopology

this = "3"
next = "1"
servername = "example.chainTopology." + this
daemon = Pyro4.core.Daemon()
obj = chainTopology.Chain(this, next)
uri = daemon.register(obj)
ns = Pyro4.locateNS()
ns.register(servername, uri)
# enter the service loop.
print("server_%s started " % this)
daemon.requestLoop()
  """
# server_3.py
with open('/content/server_3.py', 'w') as f:
  f.write(server_3.__doc__)

def chainTopology():
  """
from __future__ import print_function
import Pyro4

@Pyro4.expose
class Chain(object):
    def __init__(self, name, next):
        self.name = name
        self.nextName = next
        self.next = None

    def process(self, message):
        if self.next is None:
            self.next = Pyro4.core.Proxy("PYRONAME:example.chainTopology." + self.nextName)
        if self.name in message:
            print("Back at %s; the chain is closed!" % self.name)
            return ["complete at " + self.name]
        else:
            print("%s forwarding the message to the object %s" % (self.name, self.nextName))
            message.append(self.name)
            result = self.next.process(message)
            result.insert(0, "passed on from " + self.name)
            return result
            
  """
# chainTopology.py 
with open('/content/chainTopology.py', 'w') as f:
  f.write(chainTopology.__doc__)

In [28]:
!timeout -s KILL 5 python -m Pyro4.naming & \
timeout -s KILL 5 python server_1.py & \
timeout -s KILL 5 python server_2.py & \
timeout -s KILL 5 python server_3.py & \
timeout -s KILL 5 python client.py


Not starting broadcast server for localhost.
NS running on localhost:9090 (127.0.0.1)
URI = PYRO:Pyro.NameServer@localhost:9090
server_2 started 
server_3 started 
server_1 started 
1 forwarding the message to the object 2
2 forwarding the message to the object 3
3 forwarding the message to the object 1
Back at 1; the chain is closed!
Result=['passed on from 1', 'passed on from 2', 'passed on from 3', 'complete at 1']


**Developing a client-server application with
Pyro4**